### Union Secop I- Secop II
Secop consists of 2 platforms, Secop I and Secop II, where information for every public contract is kept; they have somme common information but differs by number of columns, column names and formatting. This notebook unifies the both datasets, filters real contracted projects and many more; necessary thing for be able to make models with all available contracting data.

In [ ]:
import pandas as pd
import requests
import numpy as np
from unidecode import unidecode

pd.options.display.max_columns = None
from sodapy import Socrata
import itertools

In [ ]:
# SECOP I
url_secop_i = 'https://www.datos.gov.co/resource/nuxh-53y2.json'
url_secop_ii = "https://www.datos.gov.co/resource/jbjy-vk9h.json"
url_secop_ii_procesos_contrat = 'https://www.datos.gov.co/resource/p6dx-8zbt.json'
url_secop_ii_proveedores = 'https://www.datos.gov.co/resource/qmzu-gj57.json'

In [ ]:
secop_i_cols = ['UID', 'Anno Cargue SECOP', 'Anno Firma del Contrato', 'Nivel Entidad',
       'Orden Entidad', 'Nombre de la Entidad', 'NIT de la Entidad',
       'Código de la Entidad', 'ID Tipo de Proceso', 'Tipo de Proceso',
       'Estado del Proceso', 'Causal de Otras Formas de Contratacion Directa',
       'ID Regimen de Contratacion', 'Regimen de Contratacion',
       'ID Objeto a Contratar', 'Objeto a Contratar',
       'Detalle del Objeto a Contratar', 'Tipo de Contrato',
       'Municipio Obtencion', 'Municipio Entrega', 'Municipios Ejecucion',
       'Fecha de Cargue en el SECOP', 'Numero de Constancia',
       'Numero de Proceso', 'Numero del Contrato', 'Cuantia Proceso',
       'ID Grupo', 'Nombre Grupo', 'ID Familia', 'Nombre Familia', 'ID Clase',
       'Nombre Clase', 'ID Ajudicacion', 'Tipo Identifi del Contratista',
       'Identificacion del Contratista', 'Nom Raz Social Contratista',
       'Dpto y Muni Contratista', 'Tipo Doc Representante Legal',
       'Identific del Represen Legal', 'Nombre del Represen Legal',
       'Fecha de Firma del Contrato', 'Fecha Ini Ejec Contrato',
       'Plazo de Ejec del Contrato', 'Rango de Ejec del Contrato',
       'Tiempo Adiciones en Dias', 'Tiempo Adiciones en Meses',
       'Fecha Fin Ejec Contrato', 'Compromiso Presupuestal',
       'Cuantia Contrato', 'Valor Total de Adiciones',
       'Valor Contrato con Adiciones', 'Objeto del Contrato a la Firma',
       'ID Origen de los Recursos', 'Origen de los Recursos', 'Codigo BPIN',
       'Proponentes Seleccionados', 'Calificacion Definitiva',
       'ID Sub Unidad Ejecutora', 'Nombre Sub Unidad Ejecutora',
       'Ruta Proceso en SECOP I', 'Moneda', 'EsPostConflicto',
       'Marcacion Adiciones', 'Posicion Rubro', 'Nombre Rubro', 'Valor Rubro',
       'Sexo RepLegal Entidad', 'Pilar Acuerdo Paz', 'Punto Acuerdo Paz',
       'Municipio Entidad', 'Departamento Entidad']

secop_ii_cols = ['Nombre Entidad', 'Nit Entidad', 'Departamento', 'Ciudad',
       'Localización', 'Orden', 'Sector', 'Rama', 'Entidad Centralizada',
       'Proceso de Compra', 'ID Contrato', 'Referencia del Contrato',
       'Estado Contrato', 'Codigo de Categoria Principal',
       'Descripcion del Proceso', 'Tipo de Contrato',
       'Modalidad de Contratacion', 'Justificacion Modalidad de Contratacion',
       'Fecha de Firma', 'Fecha de Inicio del Contrato',
       'Fecha de Fin del Contrato', 'Fecha de Inicio de Ejecucion',
       'Fecha de Fin de Ejecucion', 'Condiciones de Entrega',
       'TipoDocProveedor', 'Documento Proveedor', 'Proveedor Adjudicado',
       'Es Grupo', 'Es Pyme', 'Habilita Pago Adelantado', 'Liquidación',
       'Obligación Ambiental', 'Obligaciones Postconsumo', 'Reversion',
       'Valor del Contrato', 'Valor de pago adelantado', 'Valor Facturado',
       'Valor Pendiente de Pago', 'Valor Pagado', 'Valor Amortizado',
       'Valor Pendiente de Amortizacion', 'Valor Pendiente de Ejecucion',
       'Estado BPIN', 'Código BPIN', 'Anno BPIN', 'Saldo CDP',
       'Saldo Vigencia', 'EsPostConflicto', 'URLProceso', 'Destino Gasto',
       'Origen de los Recursos', 'Días Adicionados', 'Puntos del Acuerdo',
       'Pilares del Acuerdo']

## columns to keep on SECOPI dataset
var_ll = [
'Nombre Entidad',
'Nit Entidad',
'Departamento',
'Ciudad',
'Orden',
'Sector',
'ID Contrato',
'Estado Contrato',
'Codigo de Categoria Principal',
'Descripcion del Proceso',
'Tipo de Contrato',
'Modalidad de Contratacion',
'Fecha de Firma',
'Fecha de Inicio de Ejecucion',
'Fecha de Fin de Ejecucion',
'TipoDocProveedor',
'Documento Proveedor',
'Proveedor Adjudicado',
'Valor del Contrato',
'EsPostConflicto',
'URLProceso',
'Origen de los Recursos']

## columns to keep on SECOPII dataset
var_l = [
'Nombre de la Entidad',
'NIT de la Entidad',
'Departamento Entidad',
'Municipio Entidad',
'Orden Entidad',
'ID Ajudicacion',
'Estado del Proceso',
'ID Clase',
'Detalle del Objeto a Contratar',
'Tipo de Contrato',
'Tipo de Proceso',
'Fecha de Firma del Contrato',
'Fecha Ini Ejec Contrato',
'Fecha Fin Ejec Contrato',
'Tipo Identifi del Contratista',
'Identificacion del Contratista',
'Nom Raz Social Contratista',
'Valor Contrato con Adiciones',
'Moneda',
'EsPostConflicto',
'Ruta Proceso en SECOP I',
'Origen de los Recursos']

#Final form for built secop dataset
secop_cols = ['nombre_entidad', 'nit_entidad', 'departamento', 'ciudad', 'orden',
       'sector', 'id_contrato', 'estado_contrato',
       'codigo_de_categoria_principal', 'descripcion_del_proceso',
       'tipo_de_contrato', 'modalidad_de_contratacion', 'fecha_de_firma',
       'fecha_de_inicio_de_ejecucion', 'fecha_de_fin_de_ejecucion',
       'tipodocproveedor', 'documento_proveedor', 'proveedor_adjudicado',
       'valor_del_contrato', 'espostconflicto', 'urlproceso',
       'origen_de_los_recursos', 'moneda']

# filter invalid contracts
estado_contrato = ['27 - Otros servicios',
                   'Activo',
                   'Cedido',
                   'Cerrado',
                   'En aprobación',
                   'Modificado',
                   'Prorrogado',
                   'Suspendido',
                   'terminado']

# filter invalid contracts
estado_proceso = ['Celebrado',
                  'Liquidado',
                  'Terminado sin Liquidar']

#Columns name standarizing between secopI and secopII
columns_rename = {'Nombre de la Entidad'           :   'Nombre Entidad',
                  'NIT de la Entidad'              :   'Nit Entidad',
                  'Departamento Entidad'           :   'Departamento',
                  'Municipio Entidad'              :   'Ciudad',
                  'Orden Entidad'                  :   'Orden', 
                  'ID Ajudicacion'                 :   'ID Contrato',
                  'Estado del Proceso'             :   'Estado Contrato',
                  'ID Clase'                       :   'Codigo de Categoria Principal',
                  'Detalle del Objeto a Contratar' :   'Descripcion del Proceso',
                  'Tipo de Proceso'                :   'Modalidad de Contratacion', 
                  'Fecha de Firma del Contrato'    :   'Fecha de Firma',
                  'Fecha Ini Ejec Contrato'        :   'Fecha de Inicio de Ejecucion', 
                  'Fecha Fin Ejec Contrato'        :   'Fecha de Fin de Ejecucion',
                  'Tipo Identifi del Contratista'  :   'TipoDocProveedor', 
                  'Identificacion del Contratista' :   'Documento Proveedor',
                  'Nom Raz Social Contratista'     :   'Proveedor Adjudicado', 
                  'Valor Contrato con Adiciones'   :   'Valor del Contrato', 
                  'Ruta Proceso en SECOP I'        :   'URLProceso'}


#final form of colum names
estandarize_list = ['nombre_entidad','departamento', 'ciudad', 'orden', 'sector',
                    'estado_contrato','descripcion_del_proceso', 'tipo_de_contrato',
                    'modalidad_de_contratacion','tipodocproveedor',
                    'proveedor_adjudicado', 'origen_de_los_recursos']



In [ ]:
#dictionary for finding group from 
temp_dict =  {'A': [10],
                 'B':[11,12,13,14,15],
                 'C':[20,21,22,23,24,25,26,27],
                 'D':[30,31,32,29,39,40,41],
                 'E':[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 60],
                 'F':[70, 71, 72, 73, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 90, 91, 92, 93, 94],
                 'G':[95]}
cod_cat_princ_dict = {}
for key, value in temp_dict.items():
    for val in  value:
        cod_cat_princ_dict[val] = key

In [ ]:

def clean_secop_i(temp_df1):
    
#     temp_df1.columns = secop_i_cols
    temp_df1 = temp_df1[var_l]
    temp_df1 = temp_df1[temp_df1['Estado del Proceso'].isin(estado_proceso)]
    temp_df1 = temp_df1.rename(columns = columns_rename)
    # ya que se quiere conocer el código ese se quitan los valores invalidos
    temp_df1['Codigo de Categoria Principal'] = temp_df1['Codigo de Categoria Principal'].replace({0:np.nan})
    temp_df1 = temp_df1[temp_df1['Codigo de Categoria Principal'].notna()]
    temp_df1['Sector']= np.nan
    return temp_df1
                         
def clean_secop_ii(temp_df2):
#     temp_df2.columns = secop_ii_cols
    temp_df2 = temp_df2[var_ll]
    temp_df2 = temp_df2[temp_df2['Estado Contrato'].isin(estado_contrato)]
    temp_df2['Codigo de Categoria Principal'] = temp_df2['Codigo de Categoria Principal'].apply(lambda x: x[-8:-2])
    return temp_df2

def clean_secop(secop):
    secop = secop.reset_index()
    secop = secop.drop('index', axis = 1)
    secop.columns = secop.columns.str.lower() 
    secop.columns = secop.columns.str.replace(' ', '_')
    secop.fillna('sin_info',inplace=True)
    for col in estandarize_list:
        secop[col] = [unidecode(x) for x in secop[col]]
    secop['departamento'] = secop['departamento'].replace(['Bogota D.C.','Distrito Capital de Bogota' ],'Bogota')
    secop['departamento'] = secop['departamento'].replace(['Colombia'],'No Definido')
    secop['departamento'] = secop['departamento'].replace(['San Andres, Providencia y Santa Catalina'],'San Andres')
    secop['nit'] = secop['nit_entidad'].apply(lambda x: np.where( len(str(x))>9, str(x)[:9], str(x)) )
    secop = secop.drop('nit_entidad', axis = 1)
    secop['codigo_grupo'] = secop.codigo_de_categoria_principal.astype(str).apply(lambda x: x[:2]).astype(int).replace(cod_cat_princ_dict)
    return secop

In [ ]:
secop_1 = pd.read_csv('datos/SECOP_I_Contratos.csv', error_bad_lines=False, chunksize=700000)
secop_2 = pd.read_csv('datos/SECOP_II_Contratos.csv', error_bad_lines=False, chunksize=700000)

In [ ]:
#
for i,temp_dfs in enumerate(itertools.zip_longest(secop_1,secop_2)):
    print(i)
    temp_df1, temp_df2 = temp_dfs
    print(type(temp_df2))
    temp_df1 = temp_df1.drop('Ultima Actualizacion', axis = 1)
    
    temp_df1 = clean_secop_i(temp_df1)
    if type(temp_df2) == type(pd.DataFrame()):
        temp_df2 = clean_secop_ii(temp_df2)
        temp_secop = pd.concat([temp_df1, temp_df2])
    else:
        temp_secop = temp_df1
#     temp_df2 = clean_secop_ii(temp_df2)
#     temp_secop = pd.concat([temp_df1, temp_df2])
#     print('temp_secop ',temp_secop.columns)
    temp_secop = clean_secop(temp_secop)
    if i ==0:
        temp_secop.to_csv('datos/secop.csv', mode = 'a', index = False)
        continue
    temp_secop.to_csv('datos/secop.csv', mode = 'a', index = False, header = None)
    
# for temp_df2 in secop_2:
#     temp_df2 = clean_secop_ii(temp_df2)
#     temp_df2 = clean_secop(temp_df2)
#     temp_df2.to_csv('datos/secop.csv', mode = 'a', index = False, header = None)
#     break

In [ ]:
# Creates a smaller random sample of unified dataset.
import random
n = 6147354 #number of records in file
s = 150000 #desired sample size

skip = sorted(random.sample(range(1,n+1),n-s))
df = pd.read_csv('datos/secop.csv',skiprows=skip)
df.to_csv('datos/secop_sample.csv', index = None)


In [ ]:


# Código para traerse ano por ano los datos de la api de secop, no sirve para el 2016 y demora mucho
# for year in range(2011,2014):
#     print(year)
#     p_secop_i = {'$limit': '30000000',
#             'anno_firma_del_contrato': str(year)}
#     r_secop_i = requests.get(url_secop_i, params=p_secop_i)
#     d_secop_i = r_secop_i.json()  # To .json
#     temp_df1 = pd.DataFrame(d_secop_i)  # To df
#     print(len(temp_df1.columns))
#     if len(temp_df1.columns) == len(secop_i_cols):
#         temp_df1 = clean_secop_i(temp_df1)
        
#     p_secop_ii = {'$limit': '30000000'}
#     date = "?$where=fecha_de_firma between '{}-01-01' and '{}-01-01'".format(year,year+1)
#     r_secop_ii = requests.get(url_secop_ii+date, params=p_secop_ii)
#     d_secop_ii = r_secop_ii.json()  # To .json
#     temp_df2 = pd.DataFrame(d_secop_ii)  # To df
#     print(len(temp_df2.columns))
#     if len(temp_df2.columns) == len(secop_ii_cols):
#         temp_df2 = clean_secop_ii(temp_df2)
    
#     if (len(temp_df1.columns) == 23) and (len(temp_df2.columns) == 22):
        
# #         secop = pd.concat([temp_df1,temp_df2])
#         temp_df2 = clean_secop(temp_df2)
#         temp_df2.to_csv('datos/secop.csv', mode = 'a', index = False, header = False)
#         del temp_df2
#         print(1)
#         temp_df1 = clean_secop(temp_df1)
#         temp_df1.to_csv('datos/secop.csv', mode = 'a', index = False, header = False)
#         del temp_df1
#         continue

    
#     if (len(temp_df1.columns) == 23) and (len(temp_df2.columns) != 22):
#         print(2)
#         temp_df1 = clean_secop(temp_df1)
#         temp_df1.to_csv('datos/secop.csv', mode = 'a', index = False, header = False)
    
# #     if (len(temp_df1.columns) != 23) and (len(temp_df2.columns) == 22):
# #         print(3)
# #         secop = temp_df2
# #         secop = clean_secop(secop)
    
# #     if (len(temp_df1.columns) != 23) and (len(temp_df2.columns) != 22):
# #         print(4)
# #         continue
        
#     try:
#         del temp_df1
#         del temp_df2
#     except:
#         continue
